# EXPLORATION_4. 작사가 인공지능 만들기
## 프로젝트: 멋진 작사가 만들기


### Step 1. 데이터 다운로드
이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성했으므로 넘어 간다.

### Step 2. 데이터 읽어오기
- glob모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이하다.
- glob를 활용해서 txt파일을 읽어온 후, raw_corous 리스트에 문장 단위로 저장!

In [2]:
! mkdir -p ~/aiffel/lyricist/models
! ln -s ~/data ~/aiffel/lyricist/data

ln: failed to create symbolic link '/aiffel/aiffel/lyricist/data/data': Read-only file system


In [3]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
print(f'txt_file_path에 속한 총 파일의 개수는 {len(txt_list)} 입니다.\n')
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("\nExamples:\n", raw_corpus[:3])


txt_file_path에 속한 총 파일의 개수는 49 입니다.

데이터 크기: 187088

Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


### Step 3. 데이터 정제
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하기!!
- preprocess_sentence() 함수이용
- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거한다.
- 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있다. 그렇기 때문에 이번에는 문장을 토큰화 했을 때 토근의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기


In [5]:

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   #- 길이가 0인 문장 건너뛰기
    if sentence[-1] == ":": continue  #- 문장의 끝이 : 인 문장 건너뛰기

    if idx > 3: break   #- 우선 문장 15개 확인
        
    print(sentence)

At first I was afraid
I was petrified
I kept thinking I could never live without you
By my side But then I spent so many nights


In [6]:
import re

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [8]:
# 여기에 정제된 문장을 모을 겁니다.
corpus = []

for sentence in raw_corpus:
    #우리가 원하지 않는 문장은 건너뜁니다.
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    #정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#정제된 결과를 10개만 확인해보죠
corpus[:3]
        

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>']

**1)데이터 토큰화**
학습을 하기 위해서는 배우고자 하는 언어를 모국어로 표현을 해줘야 공부를 할 수 있다. 만약 인공지능이 학습을 하려면 언어를 숫자로 바꿔줘야 합니다. 
즉 가르칠 언어(데이터)를 숫자로 변환을 해서 인공지능에게 줄 것이다. 

tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 한 방에 해준다. 
이 과정을 벡터화(vectorize)라 하며, 숫자로 변환된 데이터를 텐서(tensor)라고 칭한다.  우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것이다.

In [19]:
import tensorflow as tf
import numpy as np

# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=15, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus) 

[[   2   71  241 ...    0    0    0]
 [   2    5   57 ...    0    0    0]
 [   2    5 1094 ...    0    0    0]
 ...
 [   2   48   16 ...    0    0    0]
 [  25    9 2855 ...  264   19    3]
 [   2    6  180 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe0fe52f390>


In [13]:
print(tensor[:3, :10])
# tokenizer에 구축된 단어 사전의 인덱스이다

[[   2   71  241    5   57  664    3    0    0    0]
 [   2    5   57 6481    3    0    0    0    0    0]
 [   2    5 1094  534    5  104   79  205  258    7]]


In [20]:
#단어사전 구축된 것 확인하기
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


위 결과 2번 인덱스가 <start>이고, 모든 행이 2로 시작하는 이유기도 하다.

In [21]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  71 241   5  57 664   3   0   0   0   0   0   0   0]
[ 71 241   5  57 664   3   0   0   0   0   0   0   0   0]


In [22]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### Step 4. 평가 데이터셋 분리
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리

In [24]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


### Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하기!

In [25]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [26]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.58596929e-04, -8.80147491e-05, -1.60654145e-05, ...,
          2.60705157e-04,  4.78841066e-05, -6.47653869e-05],
        [-2.02247684e-04, -5.62581350e-04, -3.52502888e-04, ...,
          3.76184267e-04,  2.45584553e-04, -7.16199211e-05],
        [-1.16050542e-04, -8.35636398e-04, -4.47477069e-04, ...,
          5.16100496e-04,  1.47693019e-04, -2.22521063e-04],
        ...,
        [ 3.82780272e-04, -8.45633273e-04,  5.16769593e-04, ...,
         -4.64679528e-04,  1.22320768e-03, -4.32455097e-04],
        [ 4.60603449e-04, -5.04891330e-04,  4.63683245e-04, ...,
         -3.32539057e-04,  1.30711438e-03, -5.40989859e-04],
        [ 5.05319797e-04, -8.58070707e-05,  4.84549470e-04, ...,
         -1.39514756e-04,  1.25955604e-03, -7.04170961e-04]],

       [[-1.58596929e-04, -8.80147491e-05, -1.60654145e-05, ...,
          2.60705157e-04,  4.78841066e-05, -6.47653869e-05],
        [-3.46314802e-04, -1.64598750e-04, -1

In [27]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [28]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

#결론 : 오답률이 감소하고 있으니 학습이 잘 진행되고 있다 고 해석할 수 있다. 

Epoch 1/10
686/686 [==============================] - 220s 318ms/step - loss: 4.1125
Epoch 2/10
686/686 [==============================] - 221s 321ms/step - loss: 3.1707
Epoch 3/10
686/686 [==============================] - 222s 323ms/step - loss: 2.9605
Epoch 4/10
686/686 [==============================] - 223s 325ms/step - loss: 2.8050
Epoch 5/10
686/686 [==============================] - 224s 326ms/step - loss: 2.6674
Epoch 6/10
686/686 [==============================] - 223s 325ms/step - loss: 2.5523
Epoch 7/10
686/686 [==============================] - 223s 324ms/step - loss: 2.4393
Epoch 8/10
686/686 [==============================] - 223s 325ms/step - loss: 2.3405
Epoch 9/10
686/686 [==============================] - 223s 325ms/step - loss: 2.2441
Epoch 10/10
686/686 [==============================] - 222s 324ms/step - loss: 2.1504


### Step 6.잘 만들어졌는지 평가하기
#### 문자 Test

In [29]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

### Step 7.다양한 문장 Test

In [31]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [32]:
generate_text(model, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round <end> '

In [33]:
generate_text(model, tokenizer, init_sentence="<start> i was", max_len=20)

'<start> i was the one <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> want", max_len=20)

'<start> want to be a girl with a <unk> <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> first", max_len=20)

'<start> first time i m not a girl <end> '

In [37]:
generate_text(model, tokenizer, init_sentence="<start> boy", max_len=20)

'<start> boy , you re my everything <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love me , love me , love me , love me <end> '

In [39]:
generate_text(model, tokenizer, init_sentence="<start> that", max_len=20)

'<start> that s what i m sayin <end> '

In [40]:
generate_text(model, tokenizer, init_sentence="<start> to you", max_len=20)

'<start> to you and i <end> '

### Step 8. 최종 평가
- test_loss : 2.1504
- 1epoch당 5~6분정도 소요되었고, 학습을 진행할 수록 loss률이 줄어드는 것을 확인 해볼 수 있었다.
- 데이터 정제에 따라서 결과가 달라지는 것도 알 수 있었다.
- 작문테스트: 전체적으로 자연스럼게 문장을 만들어 낸다. 

### Step 9. 정리
데이터 정제에 따라서 loss값이 더 떨어질 것으로 예상이 되는데 데이터 정제에 대한 부분을 조금더 공부를 해야 겠다는 생각 했다.
이번 노드를 하면서 인공지능으로 어디까지 만들 수 있을 까에 대한 궁금증이 생겼다. 단어를 입력하면 다음 제시 될 단어를 예측해 언어를 생성해낸 다는 것이 신기했다. 한편으로는 점점 인공지능이 사람이 대신할 부분이 많아지고 있다는 것이 조금은 불안함을 느끼기도 했다. 이러한 인공지능을 앞으로 어떻게 씌이냐도 중요할 것 같다는 생각을 하게 되었다. 코드를 어떻게 작성해야되는지 어떻게 쓰이는지 조금더 공부를 해야될 것 같다.
